In [3]:
### feature 대상으로 LGBM 학습 with Autoencoder Enhancement
# (Developed from dacon_etri_base_mod1.ipynb)

from datetime import datetime

# 현재 날짜 및 시간 가져오기
now = datetime.now()
timestamp = now.strftime("%m%d_%H%M")  # 예: 0517_1530

submission_folder = '/users/KTL/Desktop/dacon/submission/'
submission_file = f'submission_autoencoder_final_mod4_1_6_{timestamp}.csv'

In [4]:
# =========================== 라이브러리 임포트 ===========================

import numpy as np 
import pandas as pd 
import glob 
import random 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
import ast 
import warnings
warnings.filterwarnings('ignore') 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import torch
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from collections import Counter
from functools import reduce
import math

In [5]:
# =========================== 시드 고정 ===========================

# seed 고정 
SD = 42 
random.seed(SD) 
np.random.seed(SD) 
os.environ['PYTHONHASHSEED'] = str(SD)
tf.random.set_seed(SD)  # TensorFlow 시드 설정

In [6]:

# =========================== 파일 경로 설정 ===========================

# 파일 경로 설정 - VSCode 상대경로로 변경
# 실제 경로에 맞게 수정 필요
base_folder =  '/users/KTL/Desktop/ETRI_lifelog_dataset'
folder = '/ch2025_data_items'

data_dir = base_folder + folder 

# Parquet 파일 전체 경로 리스트 
parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet')) 


In [7]:

# =========================== 데이터 로딩 ===========================

# 파일 이름을 키로, DataFrame을 값으로 저장할 딕셔너리 
lifelog_data = {} 

# 파일별로 읽기 
for file_path in parquet_files: 
    name = os.path.basename(file_path).replace('.parquet', '').replace('ch2025_', '') 
    lifelog_data[name] = pd.read_parquet(file_path) 
    print(f"✅ Loaded: {name}, shape = {lifelog_data[name].shape}") 

# 딕셔너리에 있는 모든 항목을 독립적인 변수로 할당 
for key, df in lifelog_data.items(): 
    globals()[f"{key}_df"] = df 

# 메트릭스 파일 읽기
metrics_train = pd.read_csv(base_folder + '/ch2025_metrics_train.csv')
sample_submission = pd.read_csv(base_folder+'/ch2025_submission_sample.csv')

# ✅ 기준 쌍 (subject_id, lifelog_date) 
sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']) 
test_keys = set(zip(sample_submission['subject_id'], sample_submission['lifelog_date'].dt.date)) 

# ✅ DataFrame 별 timestamp 컬럼 수동 지정 
dataframes = { 
    'mACStatus': (mACStatus_df, 'timestamp'), 
    'mActivity': (mActivity_df, 'timestamp'), 
    'mAmbience': (mAmbience_df, 'timestamp'), 
    'mBle': (mBle_df, 'timestamp'), 
    'mGps': (mGps_df, 'timestamp'), 
    'mLight': (mLight_df, 'timestamp'), 
    'mScreenStatus': (mScreenStatus_df, 'timestamp'), 
    'mUsageStats': (mUsageStats_df, 'timestamp'), 
    'mWifi': (mWifi_df, 'timestamp'), 
    'wHr': (wHr_df, 'timestamp'), 
    'wLight': (wLight_df, 'timestamp'), 
    'wPedo': (wPedo_df, 'timestamp'), 
} 

# ✅ 분리 함수 
def split_test_train(df, subject_col='subject_id', timestamp_col='timestamp'): 
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce') 
    df = df.dropna(subset=[timestamp_col]) 
    df['date_only'] = df[timestamp_col].dt.date 
    df['key'] = list(zip(df[subject_col], df['date_only'])) 
    test_df = df[df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    train_df = df[~df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    return test_df, train_df 

# ✅ 결과 저장 
for name, (df, ts_col) in dataframes.items(): 
    print(f"⏳ {name} 분리 중...") 
    test_df, train_df = split_test_train(df.copy(), subject_col='subject_id', timestamp_col=ts_col) 
    globals()[f"{name}_test"] = test_df 
    globals()[f"{name}_train"] = train_df 
    print(f"✅ {name}_test → {test_df.shape}, {name}_train → {train_df.shape}") 


✅ Loaded: mACStatus, shape = (939896, 3)
✅ Loaded: mActivity, shape = (961062, 3)
✅ Loaded: mAmbience, shape = (476577, 3)
✅ Loaded: mBle, shape = (21830, 3)
✅ Loaded: mGps, shape = (800611, 3)
✅ Loaded: mLight, shape = (96258, 3)
✅ Loaded: mScreenStatus, shape = (939653, 3)
✅ Loaded: mUsageStats, shape = (45197, 3)
✅ Loaded: mWifi, shape = (76336, 3)
✅ Loaded: wHr, shape = (382918, 3)
✅ Loaded: wLight, shape = (633741, 3)
✅ Loaded: wPedo, shape = (748100, 9)
⏳ mACStatus 분리 중...
✅ mACStatus_test → (335849, 3), mACStatus_train → (604047, 3)
⏳ mActivity 분리 중...
✅ mActivity_test → (343579, 3), mActivity_train → (617483, 3)
⏳ mAmbience 분리 중...
✅ mAmbience_test → (170453, 3), mAmbience_train → (306124, 3)
⏳ mBle 분리 중...
✅ mBle_test → (8140, 3), mBle_train → (13690, 3)
⏳ mGps 분리 중...
✅ mGps_test → (287386, 3), mGps_train → (513225, 3)
⏳ mLight 분리 중...
✅ mLight_test → (34439, 3), mLight_train → (61819, 3)
⏳ mScreenStatus 분리 중...
✅ mScreenStatus_test → (336160, 3), mScreenStatus_train → (60349

In [8]:
# =========================== FEATURE ENGINEERING ===========================

def process_mACStatus(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df = df.sort_values(['subject_id', 'timestamp']) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_charging'].values # 0/1 상태 
        times = group['timestamp'].values # 충전 상태 비율 
        ratio_charging = status.mean() 
        # 상태 전이 횟수 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        lengths = [] 
        current_len = 0 
        for val in status: 
            if val == 1: 
                current_len += 1 
            elif current_len > 0: 
                lengths.append(current_len) 
                current_len = 0 
        if current_len > 0: 
            lengths.append(current_len) 
        avg_charging_duration = np.mean(lengths) if lengths else 0 
        max_charging_duration = np.max(lengths) if lengths else 0 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'charging_ratio': ratio_charging, 
            'charging_transitions': transitions, 
            'avg_charging_duration': avg_charging_duration, 
            'max_charging_duration': max_charging_duration, 
        }) 
    return pd.DataFrame(results) 

def process_mActivity(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        counts = group['m_activity'].value_counts(normalize=True) # 비율 
        row = {'subject_id': subj, 'date': date} 
        # 0~8 비율 저장 
        for i in range(9): 
            row[f'activity_{i}_ratio'] = counts.get(i, 0) 
        # 주요 활동 정보 
        row['dominant_activity'] = group['m_activity'].mode()[0] 
        row['num_unique_activities'] = group['m_activity'].nunique() 
        summary.append(row) 
    return pd.DataFrame(summary) 

# 지정된 10개 라벨 
top_10_labels = [ 
    "Inside, small room", "Speech", "Silence", "Music", "Narration, monologue", 
    "Child speech, kid speaking", "Conversation", "Speech synthesizer", "Shout", "Babbling" 
] 

def process_mAmbience_top10(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    # 초기화 
    for label in top_10_labels + ['others']: 
        df[label] = 0.0 
    for idx, row in df.iterrows(): 
        parsed = ast.literal_eval(row['m_ambience']) if isinstance(row['m_ambience'], str) else row['m_ambience'] 
        others_prob = 0.0 
        for label, prob in parsed: 
            prob = float(prob) 
            if label in top_10_labels: 
                df.at[idx, label] = prob 
            else: 
                others_prob += prob 
        df.at[idx, 'others'] = others_prob 
    return df.drop(columns=['m_ambience']) 

def summarize_mAmbience_daily(df): 
    prob_cols = [col for col in df.columns if col not in ['subject_id', 'timestamp', 'date']] 
    # 하루 단위로 평균값 요약 
    daily_summary = df.groupby(['subject_id', 'date'])[prob_cols].mean().reset_index() 
    return daily_summary 

def process_mBle(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        entry = ast.literal_eval(row['m_ble']) if isinstance(row['m_ble'], str) else row['m_ble'] 
        rssi_list = [] 
        class_0_cnt = 0 
        class_other_cnt = 0 
        for device in entry: 
            try: 
                rssi = int(device['rssi']) 
                rssi_list.append(rssi) 
                if str(device['device_class']) == '0': 
                    class_0_cnt += 1 
                else: 
                    class_other_cnt += 1 
            except: 
                continue # malformed record 
        feature = { 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'device_class_0_cnt': class_0_cnt, 
            'device_class_others_cnt': class_other_cnt, 
            'device_count': len(rssi_list), 
            'rssi_mean': np.mean(rssi_list) if rssi_list else np.nan, 
            'rssi_min': np.min(rssi_list) if rssi_list else np.nan, 
            'rssi_max': np.max(rssi_list) if rssi_list else np.nan, 
        } 
        features.append(feature) 
    return pd.DataFrame(features) 

def summarize_mBle_daily(df): 
    # row 단위 BLE feature 추출 
    df = process_mBle(df) 
    # 하루 단위로 cnt 합치기 
    grouped = df.groupby(['subject_id', 'date']).agg({ 
        'device_class_0_cnt': 'sum', 
        'device_class_others_cnt': 'sum', 
        'rssi_mean': 'mean', 
        'rssi_min': 'min', 
        'rssi_max': 'max', 
    }).reset_index() 
    # 총합 구해서 비율 계산 
    total_cnt = grouped['device_class_0_cnt'] + grouped['device_class_others_cnt'] 
    grouped['device_class_0_ratio'] = grouped['device_class_0_cnt'] / total_cnt.replace(0, np.nan) 
    grouped['device_class_others_ratio'] = grouped['device_class_others_cnt'] / total_cnt.replace(0, np.nan) 
    # 필요 없는 원래 cnt 컬럼 제거 
    grouped.drop(columns=['device_class_0_cnt', 'device_class_others_cnt'], inplace=True) 
    return grouped 

def process_mGps(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        gps_list = ast.literal_eval(row['m_gps']) if isinstance(row['m_gps'], str) else row['m_gps'] 
        altitudes = [] 
        latitudes = [] 
        longitudes = [] 
        speeds = [] 
        for entry in gps_list: 
            try: 
                altitudes.append(float(entry['altitude'])) 
                latitudes.append(float(entry['latitude'])) 
                longitudes.append(float(entry['longitude'])) 
                speeds.append(float(entry['speed'])) 
            except: 
                continue 
        features.append({ 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'altitude_mean': np.mean(altitudes) if altitudes else np.nan, 
            'latitude_std': np.std(latitudes) if latitudes else np.nan, 
            'longitude_std': np.std(longitudes) if longitudes else np.nan, 
            'speed_mean': np.mean(speeds) if speeds else np.nan, 
            'speed_max': np.max(speeds) if speeds else np.nan, 
            'speed_std': np.std(speeds) if speeds else np.nan, 
        }) 
    return pd.DataFrame(features) 

def process_mLight(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['hour'] = df['timestamp'].dt.hour 
    # 밤(22~05시), 낮(06~21시) 구분 
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6) 
    # 하루 단위 요약 
    daily = df.groupby(['subject_id', 'date']).agg( 
        light_mean=('m_light', 'mean'), 
        light_std=('m_light', 'std'), 
        light_max=('m_light', 'max'), 
        light_min=('m_light', 'min'), 
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()), 
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()), 
        light_night_ratio=('is_night', 'mean') # 밤 시간 측정 비율 
    ).reset_index() 
    return daily 

def process_mScreenStatus(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_screen_use'].values 
        ratio_on = status.mean() 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        durations = [] 
        current = 0 
        for val in status: 
            if val == 1: 
                current += 1 
            elif current > 0: 
                durations.append(current) 
                current = 0 
        if current > 0: 
            durations.append(current) 
        features.append({ 
            'subject_id': subj, 
            'date': date, 
            'screen_on_ratio': ratio_on, 
            'screen_on_transitions': transitions, 
            'screen_on_duration_avg': np.mean(durations) if durations else 0, 
            'screen_on_duration_max': np.max(durations) if durations else 0, 
        }) 
    return pd.DataFrame(features) 

top_apps = [ 
    'One UI 홈', '카카오톡', '시스템 UI', 'NAVER', '캐시워크', 
    '성경일독Q', 'YouTube', '통화', '메시지', '타임스프레드', 'Instagram'
] 

def process_mUsageStats(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        app_time = {app: 0 for app in top_apps} 
        others_time = 0 
        for row in group['m_usage_stats']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for entry in parsed: 
                app = entry.get('app_name') 
                time = entry.get('total_time', 0) 
                if app in top_apps: 
                    app_time[app] += int(time) 
                else: 
                    others_time += int(time) 
        feature = { 
            'subject_id': subj, 
            'date': date, 
            'others_time': others_time 
        } 
        # 각 앱별 컬럼 추가 
        feature.update({f'{app}_time': app_time[app] for app in top_apps}) 
        features.append(feature) 
    return pd.DataFrame(features) 

def process_mWifi(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        rssi_all = [] 
        for row in group['m_wifi']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for ap in parsed: 
                try: 
                    rssi = int(ap['rssi']) 
                    rssi_all.append(rssi) 
                except: 
                    continue 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'wifi_rssi_mean': np.mean(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_min': np.min(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_max': np.max(rssi_all) if rssi_all else np.nan, 
            'wifi_detected_cnt': len(rssi_all) 
        }) 
    return pd.DataFrame(results) 

def get_time_block(hour): 
    if 0 <= hour < 6: 
        return 'early_morning' 
    elif 6 <= hour < 12: 
        return 'morning' 
    elif 12 <= hour < 18: 
        return 'afternoon' 
    else: 
        return 'evening' 

def process_wHr_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            hr_all = [] 
            for row in block_group['heart_rate']: 
                parsed = ast.literal_eval(row) if isinstance(row, str) else row 
                hr_all.extend([int(h) for h in parsed if h is not None]) 
            if not hr_all: 
                continue 
            above_100 = [hr for hr in hr_all if hr > 100] 
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all) 
            block_stats[f'hr_{block}_std'] = np.std(hr_all) 
            block_stats[f'hr_{block}_max'] = np.max(hr_all) 
            block_stats[f'hr_{block}_min'] = np.min(hr_all) 
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

def process_wLight_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            lux = block_group['w_light'].dropna().values 
            if len(lux) == 0: 
                continue 
            block_stats[f'wlight_{block}_mean'] = np.mean(lux) 
            block_stats[f'wlight_{block}_std'] = np.std(lux) 
            block_stats[f'wlight_{block}_max'] = np.max(lux) 
            block_stats[f'wlight_{block}_min'] = np.min(lux) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

def process_wPedo(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = df.groupby(['subject_id', 'date']).agg({ 
        'step': 'sum', 
        'step_frequency': 'mean', 
        'distance': 'sum', 
        'speed': ['mean', 'max'], 
        'burned_calories': 'sum' 
    }).reset_index() 
    # 컬럼 이름 정리 
    summary.columns = ['subject_id', 'date', 'step_sum', 'step_frequency_mean', 'distance_sum', 'speed_mean', 'speed_max', 'burned_calories_sum'] 
    return summary 

# 모든 feature processing 실행
print("=== Feature Engineering 시작 ===")

mACStatus_df2 = process_mACStatus(mACStatus_df)
print("✅ mACStatus 처리 완료")

mActivity_df2 = process_mActivity(mActivity_df)
print("✅ mActivity 처리 완료")

mAmbience_df2= process_mAmbience_top10(mAmbience_df)
mAmbience_df2 = summarize_mAmbience_daily(mAmbience_df2)
print("✅ mAmbience 처리 완료")

mBle_df2 = summarize_mBle_daily(mBle_df)
print("✅ mBle 처리 완료")

m_Gps_df2 = process_mGps(mGps_df)
m_Gps_df2 = m_Gps_df2.groupby(['subject_id', 'date']).agg({
    'altitude_mean': 'mean',
    'latitude_std': 'mean',
    'longitude_std': 'mean',
    'speed_mean': 'mean',
    'speed_max': 'max',
    'speed_std': 'mean'
}).reset_index()
print("✅ mGps 처리 완료")

mLight_df2 = process_mLight(mLight_df)
print("✅ mLight 처리 완료")

mScreenStatus_df2 = process_mScreenStatus(mScreenStatus_df)
print("✅ mScreenStatus 처리 완료")

mUsageStats_df2 = process_mUsageStats(mUsageStats_df)
print("✅ mUsageStats 처리 완료")

mWifi_df2 = process_mWifi(mWifi_df)
print("✅ mWifi 처리 완료")

wHr_df2 = process_wHr_by_timeblock(wHr_df)
print("✅ wHr 처리 완료")

wLight_df2 = process_wLight_by_timeblock(wLight_df)
print("✅ wLight 처리 완료")

wPedo_df2 = process_wPedo(wPedo_df)
print("✅ wPedo 처리 완료")


=== Feature Engineering 시작 ===
✅ mACStatus 처리 완료
✅ mActivity 처리 완료
✅ mAmbience 처리 완료
✅ mBle 처리 완료
✅ mGps 처리 완료
✅ mLight 처리 완료
✅ mScreenStatus 처리 완료
✅ mUsageStats 처리 완료
✅ mWifi 처리 완료
✅ wHr 처리 완료
✅ wLight 처리 완료
✅ wPedo 처리 완료


In [9]:
# =========================== 데이터 병합 ===========================

df_list = [ 
    mACStatus_df2, 
    mActivity_df2,
    mAmbience_df2, 
    mBle_df2, 
    m_Gps_df2, 
    mLight_df2, 
    mScreenStatus_df2, 
    mUsageStats_df2, 
    mWifi_df2, 
    wHr_df2, 
    wLight_df2, 
    wPedo_df2 
] 

merged_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), df_list) 

# metrics_train의 lifelog_date → datetime.date 형으로 변환 
metrics_train['lifelog_date'] = pd.to_datetime(metrics_train['lifelog_date']).dt.date 

# merged_df의 date도 변환 
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date 

# 1. date 기준 정렬을 위해 metrics_train의 lifelog_date -> date로 맞추기 
metrics_train_renamed = metrics_train.rename(columns={'lifelog_date': 'date'}) 

# 2. train_df: metrics_train과 일치하는 (subject_id, date) → 라벨 포함 
train_df = pd.merge(metrics_train_renamed, merged_df, on=['subject_id', 'date'], how='inner') 

# 3. test_df: metrics_train에 없는 (subject_id, date) 
merged_keys = merged_df[['subject_id', 'date']] 
train_keys = metrics_train_renamed[['subject_id', 'date']] 
test_keys = pd.merge(merged_keys, train_keys, on=['subject_id', 'date'], how='left', indicator=True) 
test_keys = test_keys[test_keys['_merge'] == 'left_only'].drop(columns=['_merge']) 
test_df = pd.merge(test_keys, merged_df, on=['subject_id', 'date'], how='left') 

print(f"✅ 데이터 병합 완료: train_df={train_df.shape}, test_df={test_df.shape}")


✅ 데이터 병합 완료: train_df=(450, 115), test_df=(250, 108)


In [10]:

# =========================== 데이터 전처리 ===========================

# ✅ 타겟 리스트 
targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3'] 
target_multiclass = 'S1' 

# ✅ feature 준비 
X = train_df.drop(columns=['subject_id', 'sleep_date', 'date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']) 
X.fillna(0, inplace=True) # 결측값 처리 

test_X = test_df.drop(columns=['subject_id', 'date']) 
test_X.fillna(0, inplace=True) 

# 컬럼 이름에서 특수 문자 제거/변환 
def sanitize_column_names(df): 
    df.columns = ( 
        df.columns 
        .str.replace(r"[^\w]", "_", regex=True) # 특수문자 → _ 
        .str.replace(r"__+", "_", regex=True) # 연속된 _ 제거 
        .str.strip("_") # 앞뒤 _ 제거 
    ) 
    return df 

# 모든 입력에 적용 
X = sanitize_column_names(X) 
test_X = sanitize_column_names(test_X) 

# 모든 특성에 대해 모델링 진행 
X_selected = X.copy()
test_X_selected = test_X.copy()

print(f"✅ 전처리 완료: X_selected={X_selected.shape}, test_X_selected={test_X_selected.shape}")

✅ 전처리 완료: X_selected=(450, 106), test_X_selected=(250, 106)


In [ ]:
# =========================== 기본 LGBM으로 Feature Importance 구하기 ===========================

print("=== 기본 Feature Importance 추출 ===")

# LightGBM 기본 파라미터 설정
lgbm_params = {
    'n_estimators': 1000,
    'learning_rate': 0.03,
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}

# 이진 분류
binary_models_selected = {}
for col in targets_binary:
    print(f">> {col} 변수 Feature Importance 추출 중..")
    y = train_df[col]
    binary_params = lgbm_params.copy()
    binary_params['objective'] = 'binary'
    
    model = LGBMClassifier(**binary_params)
    model.fit(X_selected, y)
    binary_models_selected[col] = model

# 다중 분류 (S1)
print(">> S1 변수 Feature Importance 추출 중..")
y_multi = train_df['S1']
multiclass_params = lgbm_params.copy()
multiclass_params['num_class'] = len(y_multi.unique())

model_s1_selected = LGBMClassifier(**multiclass_params)
model_s1_selected.fit(X_selected, y_multi)

# Feature Importance 추출
feature_importance_dict = {}

# 이진분류 모델들의 feature importance
for col_name, model in binary_models_selected.items():
    importance_df = pd.DataFrame({
        'feature': X_selected.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False).head(10)  # 상위 10개
    feature_importance_dict[col_name] = importance_df.copy()

# S1 다중분류 모델의 feature importance
fi_multi_20 = pd.DataFrame({
    'feature': X_selected.columns,
    'importance': model_s1_selected.feature_importances_
}).sort_values('importance', ascending=False).head(20)
feature_importance_dict['S1'] = fi_multi_20.copy()

print("✅ Feature Importance 추출 완료")

=== 기본 Feature Importance 추출 ===
>> Q1 변수 Feature Importance 추출 중..
>> Q2 변수 Feature Importance 추출 중..
>> Q3 변수 Feature Importance 추출 중..
>> S2 변수 Feature Importance 추출 중..
>> S3 변수 Feature Importance 추출 중..
>> S1 변수 Feature Importance 추출 중..
✅ Feature Importance 추출 완료


In [ ]:
# =========================== 오토인코더 특성 추출 ===========================

print("===== 각 모델별 오토인코더를 사용한 고급 특성 추출 =====")

# 각 모델별 결과를 저장할 딕셔너리들
autoencoder_results = {}
encoder_results = {}
combined_features = {}
scalers = {}

# 모델 리스트
model_names = ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']

def create_autoencoder(input_dim, encoding_dim=32):
    """개선된 오토인코더 모델 생성"""
    # 입력 레이어
    input_layer = Input(shape=(input_dim,), name='input')
    
    # 인코더 부분 (점진적 차원 축소)
    x = Dense(min(128, input_dim * 2), activation='relu', name='encoder_1')(input_layer)
    x = BatchNormalization(name='bn_encoder_1')(x)
    x = Dropout(0.2, name='dropout_encoder_1')(x)
    
    x = Dense(min(64, input_dim), activation='relu', name='encoder_2')(x)
    x = BatchNormalization(name='bn_encoder_2')(x)
    x = Dropout(0.2, name='dropout_encoder_2')(x)
    
    # 병목층 (잠재 공간)
    encoded = Dense(encoding_dim, activation='relu', name='latent_space')(x)
    
    # 디코더 부분 (점진적 차원 복원)
    x = Dense(min(64, input_dim), activation='relu', name='decoder_1')(encoded)
    x = BatchNormalization(name='bn_decoder_1')(x)
    x = Dropout(0.2, name='dropout_decoder_1')(x)
    
    x = Dense(min(128, input_dim * 2), activation='relu', name='decoder_2')(x)
    x = BatchNormalization(name='bn_decoder_2')(x)
    x = Dropout(0.2, name='dropout_decoder_2')(x)
    
    # 출력 레이어
    decoded = Dense(input_dim, activation='sigmoid', name='output')(x)
    
    # 모델 생성
    autoencoder = Model(input_layer, decoded, name='autoencoder')
    encoder = Model(input_layer, encoded, name='encoder')
    
    return autoencoder, encoder

# 각 모델별 오토인코더 처리
for model_name in model_names:
    print(f"\n🔹 {model_name} 모델 오토인코더 처리 시작...")
    
    # 해당 모델의 상위 20개 중요 피처 선택
    top_features = feature_importance_dict[model_name]['feature'].tolist()
    
    # 중요 피처들로 데이터 필터링
    X_model_selected = X_selected[top_features]
    test_X_model_selected = test_X_selected[top_features]
    
    # 데이터 정규화 (오토인코더 성능 향상을 위해)
    scaler = StandardScaler()
    X_model_scaled = scaler.fit_transform(X_model_selected)
    test_X_model_scaled = scaler.transform(test_X_model_selected)
    
    print(f"   - 선택된 피처 수: {len(top_features)}개")
    print(f"   - 최고 중요도 피처: {feature_importance_dict[model_name].iloc[0]['feature']}")
    
    # 하이퍼파라미터 설정
    input_dim = len(top_features)
    encoding_dim = max(8, min(32, input_dim))  # 적응적 인코딩 차원
    batch_size = 64
    epochs = 150
    
    print(f"   - 입력 차원: {input_dim}개")
    print(f"   - 인코딩 차원: {encoding_dim}개")
    
    # 오토인코더 모델 생성
    autoencoder, encoder = create_autoencoder(input_dim, encoding_dim)
    
    # 모델 컴파일
    autoencoder.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    # 훈련/검증 세트 분리
    X_train_auto, X_val_auto = train_test_split(
        X_model_scaled, 
        test_size=0.2, 
        random_state=42
    )
    
    # 콜백 설정
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=0
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=8,
            min_lr=1e-6,
            verbose=0
        )
    ]
    
    # 오토인코더 모델 학습
    print(f"   - {model_name} 오토인코더 학습 중...")
    history = autoencoder.fit(
        X_train_auto, X_train_auto,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=True,
        validation_data=(X_val_auto, X_val_auto),
        callbacks=callbacks,
        verbose=0  # 출력 최소화
    )
    
    # 인코딩된 특성 추출
    print(f"   - 인코딩된 특성 추출 중...")
    encoded_features_train = encoder.predict(X_model_scaled, verbose=0)
    encoded_features_test = encoder.predict(test_X_model_scaled, verbose=0)
    
    # 원본 특성과 인코딩된 특성 결합
    X_combined_model = np.hstack([X_model_scaled, encoded_features_train])
    test_X_combined_model = np.hstack([test_X_model_scaled, encoded_features_test])
    
    # 결과 저장
    autoencoder_results[model_name] = autoencoder
    encoder_results[model_name] = encoder
    scalers[model_name] = scaler
    combined_features[model_name] = {
        'train': X_combined_model,
        'test': test_X_combined_model,
        'original_features': top_features,
        'original_shape': X_model_selected.shape,
        'encoded_shape': encoded_features_train.shape,
        'combined_shape': X_combined_model.shape,
        'scaler': scaler
    }
    
    # 최종 손실값 계산
    final_loss = min(history.history['val_loss'])
    
    print(f"   ✅ {model_name} 완료!")
    print(f"      - 원본 특성 수: {len(top_features)}개")
    print(f"      - 인코딩된 특성 수: {encoding_dim}개")
    print(f"      - 결합된 특성 수: {X_combined_model.shape[1]}개")
    print(f"      - 최종 검증 손실: {final_loss:.6f}")
    print(f"      - 학습 에포크: {len(history.history['loss'])}회")

print("\n" + "="*60)
print("🎉 모든 모델별 오토인코더 처리 완료!")
print("="*60)

===== 각 모델별 오토인코더를 사용한 고급 특성 추출 =====

🔹 Q1 모델 오토인코더 처리 시작...
   - 선택된 피처 수: 20개
   - 최고 중요도 피처: light_night_mean
   - 입력 차원: 20개
   - 인코딩 차원: 10개
   - Q1 오토인코더 학습 중...
   - 인코딩된 특성 추출 중...
   ✅ Q1 완료!
      - 원본 특성 수: 20개
      - 인코딩된 특성 수: 10개
      - 결합된 특성 수: 30개
      - 최종 검증 손실: 0.659387
      - 학습 에포크: 150회

🔹 Q2 모델 오토인코더 처리 시작...
   - 선택된 피처 수: 20개
   - 최고 중요도 피처: activity_7_ratio
   - 입력 차원: 20개
   - 인코딩 차원: 10개
   - Q2 오토인코더 학습 중...
   - 인코딩된 특성 추출 중...
   ✅ Q2 완료!
      - 원본 특성 수: 20개
      - 인코딩된 특성 수: 10개
      - 결합된 특성 수: 30개
      - 최종 검증 손실: 0.789956
      - 학습 에포크: 150회

🔹 Q3 모델 오토인코더 처리 시작...
   - 선택된 피처 수: 20개
   - 최고 중요도 피처: others_time
   - 입력 차원: 20개
   - 인코딩 차원: 10개
   - Q3 오토인코더 학습 중...
   - 인코딩된 특성 추출 중...
   ✅ Q3 완료!
      - 원본 특성 수: 20개
      - 인코딩된 특성 수: 10개
      - 결합된 특성 수: 30개
      - 최종 검증 손실: 0.703759
      - 학습 에포크: 150회

🔹 S1 모델 오토인코더 처리 시작...
   - 선택된 피처 수: 20개
   - 최고 중요도 피처: others_time
   - 입력 차원: 20개
   - 인코딩 차원: 10개
   - S1 오토인코더 학습 중...
   - 인코

In [ ]:
# =========================== 오토인코더 특성을 사용한 LightGBM 학습 ===========================

print("=" * 60)
print("🚀 오토인코더 특성을 사용한 LightGBM 모델 학습 시작")
print("=" * 60)

# 각 모델별 결과를 저장할 딕셔너리 (오토인코더 특성 사용)
binary_models_autoencoder = {}
binary_preds_autoencoder = {}
multiclass_s1_autoencoder = {}

# S1을 제외한 이진분류 모델들
binary_model_names = ['Q1', 'Q2', 'Q3', 'S2', 'S3']

# 개선된 LightGBM 파라미터 그리드
improved_binary_param_grid = {
    'learning_rate': [0.01, 0.03],
    'n_estimators': [1000, 1500],
    'num_leaves': [31, 50],
    'max_depth': [-1, 6],
    'min_child_samples': [10, 20],
    # 'subsample': [0.9, 1.0],
    # 'colsample_bytree': [0.9, 1.0],
    # 'reg_alpha': [0, 0.01, 0.1],
    # 'reg_lambda': [0, 0.01, 0.1],
}

# 기본 LightGBM 파라미터
base_lgbm_params = {
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1,
    'force_row_wise': True  # 안정성을 위해 추가
}

# 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def create_smote_pipeline(model, k_neighbors=3):
    """SMOTE + LightGBM 파이프라인 생성"""
    return ImbPipeline([
        ('smote', SMOTE(random_state=42, k_neighbors=k_neighbors)),
        ('classifier', model)
    ])

def get_optimal_k_neighbors(y):
    """클래스 분포에 따른 최적 k_neighbors 계산"""
    min_class_count = min(y.value_counts())
    if min_class_count <= 3:
        return max(1, min_class_count - 1)
    elif min_class_count <= 5:
        return 3
    else:
        return 5

def train_binary_model_with_autoencoder(model_name, target_col):
    """오토인코더 특성을 사용한 이진분류 모델 학습"""
    print(f"\n🎯 {model_name} 모델로 {target_col} 타겟 예측 (오토인코더 특성 사용)")
    print("-" * 50)
    
    # 타겟 데이터 준비
    y = train_df[target_col]
    
    # 오토인코더로 생성된 결합 특성 사용
    X_train_model = combined_features[model_name]['train']
    X_test_model = combined_features[model_name]['test']
    
    print(f"   - 사용 특성 수: {X_train_model.shape[1]}개 (오토인코더 결합)")
    print(f"   - 훈련 샘플 수: {X_train_model.shape[0]}개")
    print(f"   - 타겟: {target_col}")
    print(f"   - 클래스 수: {len(y.unique())}개")
    print(f"   - 원본 클래스 분포: {dict(y.value_counts().sort_index())}")
    
    # 최적 k_neighbors 계산
    k_neighbors = get_optimal_k_neighbors(y)
    print(f"   - SMOTE k_neighbors: {k_neighbors}")
    
    # SMOTE 적용 후 클래스 분포 미리보기
    try:
        smote_preview = SMOTE(random_state=42, k_neighbors=k_neighbors)
        X_resampled, y_resampled = smote_preview.fit_resample(X_train_model, y)
        print(f"   - SMOTE 후 클래스 분포: {dict(pd.Series(y_resampled).value_counts().sort_index())}")
        print(f"   - 샘플 수 변화: {len(y)} → {len(y_resampled)}")
    except Exception as e:
        print(f"   ⚠️  SMOTE 미리보기 실패: {str(e)}")
        k_neighbors = 1
        print(f"   - k_neighbors를 1로 재조정")
    
    # LightGBM 모델 생성
    lgbm_model = LGBMClassifier(
        objective='binary',
        **base_lgbm_params
    )
    
    # SMOTE + LightGBM 파이프라인 생성
    pipeline = create_smote_pipeline(lgbm_model, k_neighbors)
    
    # 파라미터 그리드 (classifier__ 접두사 추가)
    param_grid_pipeline = {}
    for key, value in improved_binary_param_grid.items():
        param_grid_pipeline[f'classifier__{key}'] = value
    
    # 그리드 서치
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid_pipeline,
        scoring='f1_macro',
        cv=cv,
        verbose=0,
        n_jobs=-1,
        error_score='raise'  # 에러 발생 시 명확한 메시지
    )
    
    print(f"   - 모델 최적화 중...")
    
    try:
        grid_search.fit(X_train_model, y)
        
        print(f"   ✅ 학습 완료!")
        print(f"      - 최적 F1-Macro 점수: {grid_search.best_score_:.6f}")
        
        # 예측 수행
        best_model = grid_search.best_estimator_
        predictions = best_model.predict(X_test_model)
        
        # 결과 저장
        binary_models_autoencoder[model_name] = best_model
        binary_preds_autoencoder[model_name] = predictions
        
        print(f"      - 예측 완료: {len(predictions)}개 샘플")
        print(f"      - 예측 클래스 분포: {dict(pd.Series(predictions).value_counts().sort_index())}")
        
        return True
        
    except Exception as e:
        print(f"   ❌ 학습 실패: {str(e)}")
        return False

def train_multiclass_s1_with_autoencoder():
    """오토인코더 특성을 사용한 S1 다중분류 모델 학습"""
    print(f"\n🎯 S1 모델로 S1 다중분류 타겟 예측 (오토인코더 특성 사용)")
    print("-" * 50)
    
    # S1 타겟 데이터 준비
    target_col = 'S1'
    y_multiclass = train_df[target_col]
    
    # 오토인코더로 생성된 결합 특성 사용
    X_train_s1 = combined_features[target_col]['train']
    X_test_s1 = combined_features[target_col]['test']
    
    print(f"   - 사용 특성 수: {X_train_s1.shape[1]}개 (오토인코더 결합)")
    print(f"   - 훈련 샘플 수: {X_train_s1.shape[0]}개")
    print(f"   - 타겟: {target_col} (다중분류)")
    print(f"   - 클래스 수: {len(y_multiclass.unique())}개")
    print(f"   - 원본 클래스 분포: {dict(y_multiclass.value_counts().sort_index())}")
    
    # 다중분류용 SMOTE 적용
    k_neighbors = get_optimal_k_neighbors(y_multiclass)
    print(f"   - SMOTE k_neighbors: {k_neighbors}")
    
    try:
        smote = SMOTE(random_state=42, k_neighbors=k_neighbors)
        X_train_s1_smote, y_multiclass_smote = smote.fit_resample(X_train_s1, y_multiclass)
        print(f"   - SMOTE 후 클래스 분포: {dict(pd.Series(y_multiclass_smote).value_counts().sort_index())}")
        print(f"   - 샘플 수 변화: {len(y_multiclass)} → {len(y_multiclass_smote)}")
    except Exception as e:
        print(f"   ⚠️  SMOTE 적용 실패: {str(e)}")
        X_train_s1_smote, y_multiclass_smote = X_train_s1, y_multiclass
        print(f"   - 원본 데이터로 진행")
    
    # 다중분류용 모델 설정
    multiclass_model = LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        **base_lgbm_params
    )
    
    # 다중분류용 파라미터 그리드 (간소화)
    multiclass_param_grid = {
        'learning_rate': [0.01, 0.03, 0.05],
        'n_estimators': [1000, 1500],
        'num_leaves': [31, 50, 100],
        'max_depth': [-1, 6],
        'min_child_samples': [10, 20],
    }
    
    # 그리드 서치
    grid_search_multiclass = GridSearchCV(
        estimator=multiclass_model,
        param_grid=multiclass_param_grid,
        scoring='f1_macro',
        cv=cv,
        verbose=0,
        n_jobs=-1
    )
    
    print(f"   - S1 다중분류 모델 최적화 중...")
    
    try:
        grid_search_multiclass.fit(X_train_s1_smote, y_multiclass_smote)
        
        print(f"   ✅ 학습 완료!")
        print(f"      - 최적 F1-Macro 점수: {grid_search_multiclass.best_score_:.6f}")
        
        # 예측 수행
        best_multiclass_model = grid_search_multiclass.best_estimator_
        multiclass_predictions = best_multiclass_model.predict(X_test_s1)
        multiclass_probabilities = best_multiclass_model.predict_proba(X_test_s1)
        
        # 결과 저장
        multiclass_s1_autoencoder['model'] = best_multiclass_model
        multiclass_s1_autoencoder['predictions'] = multiclass_predictions
        multiclass_s1_autoencoder['probabilities'] = multiclass_probabilities
        multiclass_s1_autoencoder['classes'] = best_multiclass_model.classes_
        
        print(f"      - 예측 완료: {len(multiclass_predictions)}개 샘플")
        print(f"      - 예측 클래스 분포: {dict(pd.Series(multiclass_predictions).value_counts().sort_index())}")
        
        return True
        
    except Exception as e:
        print(f"   ❌ 학습 실패: {str(e)}")
        return False


🚀 오토인코더 특성을 사용한 LightGBM 모델 학습 시작


In [16]:
# =========================== 모델 학습 실행 ===========================

# 이진분류 모델들 학습
print("\n📊 이진분류 모델 학습 (오토인코더 특성 사용)")
print("=" * 60)

successful_models = []
failed_models = []

for model_name in binary_model_names:
    target_col = model_name  # Q1→Q1, Q2→Q2, Q3→Q3, S2→S2, S3→S3
    success = train_binary_model_with_autoencoder(model_name, target_col)
    
    if success:
        successful_models.append(model_name)
    else:
        failed_models.append(model_name)

# S1 다중분류 모델 학습
print("\n📊 다중분류 모델 학습 (오토인코더 특성 사용)")
print("=" * 60)

s1_success = train_multiclass_s1_with_autoencoder()

# =========================== 결과 요약 ===========================

print("\n" + "=" * 60)
print("🎉 오토인코더 특성 기반 모델 학습 완료!")
print("=" * 60)

print(f"\n📊 학습 결과 요약:")
print(f"   🟢 성공한 이진분류 모델: {len(successful_models)}개")
for model in successful_models:
    print(f"      - {model}")

if failed_models:
    print(f"   🔴 실패한 이진분류 모델: {len(failed_models)}개")
    for model in failed_models:
        print(f"      - {model}")

print(f"   🟢 S1 다중분류 모델: {'성공' if s1_success else '실패'}")


📊 이진분류 모델 학습 (오토인코더 특성 사용)

🎯 Q1 모델로 Q1 타겟 예측 (오토인코더 특성 사용)
--------------------------------------------------
   - 사용 특성 수: 30개 (오토인코더 결합)
   - 훈련 샘플 수: 450개
   - 타겟: Q1
   - 클래스 수: 2개
   - 원본 클래스 분포: {0: 227, 1: 223}
   - SMOTE k_neighbors: 5
   - SMOTE 후 클래스 분포: {0: 227, 1: 227}
   - 샘플 수 변화: 450 → 454
   - 모델 최적화 중...
   ✅ 학습 완료!
      - 최적 F1-Macro 점수: 0.627922
      - 예측 완료: 250개 샘플
      - 예측 클래스 분포: {0: 119, 1: 131}

🎯 Q2 모델로 Q2 타겟 예측 (오토인코더 특성 사용)
--------------------------------------------------
   - 사용 특성 수: 30개 (오토인코더 결합)
   - 훈련 샘플 수: 450개
   - 타겟: Q2
   - 클래스 수: 2개
   - 원본 클래스 분포: {0: 197, 1: 253}
   - SMOTE k_neighbors: 5
   - SMOTE 후 클래스 분포: {0: 253, 1: 253}
   - 샘플 수 변화: 450 → 506
   - 모델 최적화 중...
   ✅ 학습 완료!
      - 최적 F1-Macro 점수: 0.593284
      - 예측 완료: 250개 샘플
      - 예측 클래스 분포: {0: 99, 1: 151}

🎯 Q3 모델로 Q3 타겟 예측 (오토인코더 특성 사용)
--------------------------------------------------
   - 사용 특성 수: 30개 (오토인코더 결합)
   - 훈련 샘플 수: 450개
   - 타겟: Q3
   - 클래스 수: 2개
   - 원본 클래스 

In [17]:
# =========================== 최종 제출 파일 생성 ===========================

print("=" * 60)
print("📋 오토인코더 기반 최종 제출 파일 생성")
print("=" * 60)

# sample 기반 제출 포맷 가져오기
submission_autoencoder = sample_submission[['subject_id', 'sleep_date', 'lifelog_date']].copy()

# lifelog_date 기준으로 string → date 형식 통일
submission_autoencoder['lifelog_date'] = pd.to_datetime(submission_autoencoder['lifelog_date']).dt.date

print(f"📊 제출 데이터 기본 정보:")
print(f"   - 제출 샘플 수: {len(submission_autoencoder):,}개")
print(f"   - 고유 피험자 수: {submission_autoencoder['subject_id'].nunique()}명")

# 예측 결과 검증 및 할당
def assign_predictions_safely(df, target_col, predictions, model_type="이진분류"):
    """안전하게 예측 결과를 할당"""
    if len(df) != len(predictions):
        print(f"   ⚠️  {target_col} 길이 불일치: submission={len(df)}, predictions={len(predictions)}")
        # 길이가 다르면 최소 길이로 맞춤
        min_length = min(len(df), len(predictions))
        df = df.iloc[:min_length].copy()
        predictions = predictions[:min_length]
        print(f"   → 길이를 {min_length}로 조정")
    
    df[target_col] = predictions.astype(int)
    
    # 예측 결과 요약
    value_counts = pd.Series(predictions).value_counts().sort_index()
    total = len(predictions)
    
    print(f"   ✅ {target_col} ({model_type}) 할당 완료:")
    for value, count in value_counts.items():
        percentage = (count / total) * 100
        print(f"      클래스 {value}: {count:,}개 ({percentage:.1f}%)")
    
    return df

# S1 다중분류 예측 결과 할당
if s1_success and 'predictions' in multiclass_s1_autoencoder:
    print(f"\n🔄 S1 다중분류 예측 결과 할당 중...")
    s1_predictions = multiclass_s1_autoencoder['predictions']
    submission_autoencoder = assign_predictions_safely(
        submission_autoencoder, 'S1', s1_predictions, "다중분류"
    )
else:
    print(f"\n❌ S1 다중분류 예측 결과가 없습니다. 기본값(0)으로 설정합니다.")
    submission_autoencoder['S1'] = 0

# 이진분류 예측 결과 할당
print(f"\n🔄 이진분류 예측 결과 할당 중...")
for col in ['Q1', 'Q2', 'Q3', 'S2', 'S3']:
    if col in binary_preds_autoencoder:
        individual_predictions = binary_preds_autoencoder[col]
        submission_autoencoder = assign_predictions_safely(
            submission_autoencoder, col, individual_predictions, "이진분류"
        )
    else:
        print(f"   ⚠️  {col} 예측 결과가 없습니다. 기본값(0)으로 설정합니다.")
        submission_autoencoder[col] = 0

# 최종 제출 형식 정렬 및 검증
submission_autoencoder = submission_autoencoder[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]

print(f"\n📊 최종 예측 결과 요약 (오토인코더 기반):")
print("=" * 50)

for col in ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']:
    value_counts = submission_autoencoder[col].value_counts().sort_index()
    total = len(submission_autoencoder)
    print(f"🔹 {col}:")
    for value, count in value_counts.items():
        percentage = (count / total) * 100
        print(f"   클래스 {value}: {count:,}개 ({percentage:.1f}%)")

# 데이터 품질 검증
print(f"\n🔍 데이터 품질 검증:")
print(f"   - 총 행 수: {len(submission_autoencoder):,}")
print(f"   - 결측값 확인:")
for col in ['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']:
    null_count = submission_autoencoder[col].isnull().sum()
    if null_count > 0:
        print(f"      {col}: {null_count}개 결측값 ⚠️")
    else:
        print(f"      {col}: 결측값 없음 ✅")

# 파일 저장
submission_path = submission_folder + submission_file
submission_autoencoder.to_csv(submission_path, index=False)

print(f"\n✅ 오토인코더 기반 제출 파일 저장 완료!")
print(f"   - 파일명: {submission_file}")
print(f"   - 저장 경로: {submission_path}")
print(f"   - 파일 크기: {len(submission_autoencoder):,} 행 x {len(submission_autoencoder.columns)} 열")

print(f"\n" + "=" * 60)
print("🎉 오토인코더 기반 모델링 프로세스 완료!")
print("=" * 60)

print(f"\n📋 완료된 작업 요약:")
print(f"   ✅ 각 모델별 상위 피처 선택 (20개씩)")
print(f"   ✅ 모델별 오토인코더 학습 및 특성 추출")
print(f"   ✅ 원본 + 인코딩 특성 결합")
print(f"   ✅ SMOTE + LightGBM 하이퍼파라미터 최적화")
print(f"   ✅ 이진분류 모델 {len(successful_models)}개 학습")
print(f"   ✅ S1 다중분류 모델 학습")
print(f"   ✅ 최종 제출 파일 생성")

print(f"\n🔍 성능 개선 포인트:")
print(f"   - 각 모델별 개별 오토인코더로 특성 최적화")
print(f"   - StandardScaler로 정규화 적용")
print(f"   - SMOTE로 클래스 불균형 해결")
print(f"   - 확장된 하이퍼파라미터 튜닝")
print(f"   - 5-fold 교차검증으로 견고한 평가")

print("=" * 60)

📋 오토인코더 기반 최종 제출 파일 생성
📊 제출 데이터 기본 정보:
   - 제출 샘플 수: 250개
   - 고유 피험자 수: 10명

🔄 S1 다중분류 예측 결과 할당 중...
   ✅ S1 (다중분류) 할당 완료:
      클래스 0: 69개 (27.6%)
      클래스 1: 142개 (56.8%)
      클래스 2: 39개 (15.6%)

🔄 이진분류 예측 결과 할당 중...
   ✅ Q1 (이진분류) 할당 완료:
      클래스 0: 119개 (47.6%)
      클래스 1: 131개 (52.4%)
   ✅ Q2 (이진분류) 할당 완료:
      클래스 0: 99개 (39.6%)
      클래스 1: 151개 (60.4%)
   ✅ Q3 (이진분류) 할당 완료:
      클래스 0: 84개 (33.6%)
      클래스 1: 166개 (66.4%)
   ✅ S2 (이진분류) 할당 완료:
      클래스 0: 81개 (32.4%)
      클래스 1: 169개 (67.6%)
   ✅ S3 (이진분류) 할당 완료:
      클래스 0: 66개 (26.4%)
      클래스 1: 184개 (73.6%)

📊 최종 예측 결과 요약 (오토인코더 기반):
🔹 Q1:
   클래스 0: 119개 (47.6%)
   클래스 1: 131개 (52.4%)
🔹 Q2:
   클래스 0: 99개 (39.6%)
   클래스 1: 151개 (60.4%)
🔹 Q3:
   클래스 0: 84개 (33.6%)
   클래스 1: 166개 (66.4%)
🔹 S1:
   클래스 0: 69개 (27.6%)
   클래스 1: 142개 (56.8%)
   클래스 2: 39개 (15.6%)
🔹 S2:
   클래스 0: 81개 (32.4%)
   클래스 1: 169개 (67.6%)
🔹 S3:
   클래스 0: 66개 (26.4%)
   클래스 1: 184개 (73.6%)

🔍 데이터 품질 검증:
   - 총 행 수: 250
   - 결측값 확인:
      Q1: 결측값